In [ ]:
import logging
import pandas as pd
import datetime
import seaborn as sb
import time
import os.path
from ipywidgets import interact, fixed
from libs.CovidTimeseriesModel import CovidTimeseriesModel
from libs.CovidDatasets import CDSDataset, JHUDataset
from libs.CovidUtil import CovidUtil
pd.set_option('display.float_format', lambda x: '%.12f' % x)

assumptions = {'r0':1.0, 
               'HOSPITALIZATION_RATE':.0727,
               'HOSPITALIZED_CASES_REQUIRING_ICU_CARE':.1397,
               'TOTAL_INFECTED_PERIOD':12,
               'MODEL_INTERVAL':4}

In [ ]:
def set_r0(r0):
    if not r0.isdigit():
        return
    assumptions['r0']=float(r0)
    return run_CA_model()
    

In [ ]:
def set_hospitalization_rate(hr):
    if not hr.isdigit():
        return
    assumptions['HOSPITALIZATION_RATE']=float(hr)
    return run_CA_model()

In [ ]:
def set_hospital_cases_requiring_ICU_care(icurate):
    if not icurate.isdigit():
        return
    assumptions['HOSPITALIZED_CASES_REQUIRING_ICU_CARE']=float(icurate)
    return run_CA_model()

In [ ]:
def set_total_infected_period(prd):
    if not prd.isdigit():
        return
    assumptions['TOTAL_INFECTED_PERIOD']=float(prd)
    return run_CA_model()

In [ ]:
def set_model_interval(interval):
    if not interval.isdigit():
        return
    assumptions['MODEL_INTERVAL']=float(interval)
    return run_CA_model()

In [ ]:
def run_CA_model():

    INTERVENTIONS = [
        None, 
        {
            datetime.date(2020, 3, 23): 1.3,
            datetime.date(2020, 4, 20): 1.1,
            datetime.date(2020, 5, 22): 0.8,
            datetime.date(2020, 6, 23): assumptions['r0']
        },
        {
            datetime.date(2020, 3, 23): 1.7,
            datetime.date(2020, 6, 23): assumptions['r0']
        },
        {
            datetime.date(2020, 3, 23): 1.3,
            datetime.date(2020, 3, 31): 0.3,
            datetime.date(2020, 4, 28): 0.2,
            datetime.date(2020, 5,  6): 0.1,
            datetime.date(2020, 5, 10): 0.35,
            datetime.date(2020, 5, 18): assumptions['r0']
        }
    ]
    df = model_state('USA', 'CA', INTERVENTIONS[0])
    return(df)

In [ ]:
def record_results(res, directory, name, num, pop):
    import copy
    import os.path
    vals = copy.copy(res)
    # Format the date in the manner the front-end expects
    vals['Date'] = res['Date'].apply(lambda d: "{}/{}/{}".format(d.month, d.day, d.year)) 
    # Set the population
    vals['Population'] = pop
    # Write the results to the specified directory
    with open( os.path.join(directory, name + '.' + str(num) + '.json').format(name), 'w') as out:
        out.write(
            str(vals[[
                'Date', 
                'R', 
                'Beg. Susceptible', 
                'New Inf.', 
                'Curr. Inf.', 
                'Recov. or Died', 
                'End Susceptible', 
                'Actual Reported', 
                'Pred. Hosp.', 
                'Cum. Inf.', 
                'Cum. Deaths', 
                'Avail. Hosp. Beds', 
                'S&P 500', 
                'Est. Actual Chance of Inf.', 
                'Pred. Chance of Inf.', 
                'Cum. Pred. Chance of Inf.', 
                'Population',
                'R0', 
                '% Susceptible'
            ]].values.tolist())
        )

In [ ]:
def model_state(country, state, interventions=None):
    ## Constants
    start_time = time.time()
    #HOSPITALIZATION_RATE = .0727
    #HOSPITALIZED_CASES_REQUIRING_ICU_CARE = .1397
    #TOTAL_INFECTED_PERIOD = 12
    #MODEL_INTERVAL = 4
    #r0 = 2.4
    r0 = assumptions['r0']
    Dataset = CDSDataset()
    #Dataset = JHUDataset()
    POP = Dataset.get_population_by_country_state(country, state)
    # Pack all of the assumptions and parameters into a dict that can be passed into the model
    MODEL_PARAMETERS = {
        # Pack the changeable model parameters
        'timeseries': Dataset.get_timeseries_by_country_state(country, state, assumptions['MODEL_INTERVAL']),
        'beds': Dataset.get_beds_by_country_state(country, state),
        'population': POP,
        'projection_iterations': 25, # Number of iterations into the future to project
        'r0': r0,
        'interventions': interventions,
        'hospitalization_rate': assumptions['HOSPITALIZATION_RATE'],
        'initial_hospitalization_rate': .05,
        'case_fatality_rate': .0109341104294479,
        'hospitalized_cases_requiring_icu_care': assumptions['HOSPITALIZED_CASES_REQUIRING_ICU_CARE'],
        # Assumes that anyone who needs ICU care and doesn't get it dies
        'case_fatality_rate_hospitals_overwhelmed': assumptions['HOSPITALIZATION_RATE'] * assumptions['HOSPITALIZED_CASES_REQUIRING_ICU_CARE'],
        'hospital_capacity_change_daily_rate': 1.05,
        'max_hospital_capacity_factor': 2.07,
        'initial_hospital_bed_utilization': .6,
        'model_interval': 4, # In days
        'total_infected_period': 12, # In days
        'rolling_intervals_for_current_infected': int(round(assumptions['TOTAL_INFECTED_PERIOD'] / assumptions['MODEL_INTERVAL'], 0)),
    }
    return CovidTimeseriesModel().forecast(model_parameters=MODEL_PARAMETERS)


interact(set_r0, 
         r0=str(assumptions['r0']))

interact(set_hospitalization_rate,
         hr=str(assumptions['HOSPITALIZATION_RATE']))

interact(set_hospital_cases_requiring_ICU_care,
         icurate=str(assumptions['HOSPITALIZED_CASES_REQUIRING_ICU_CARE']))

interact(set_total_infected_period,
         prd=str(assumptions['TOTAL_INFECTED_PERIOD']))

interact(set_model_interval,
         interval=str(assumptions['MODEL_INTERVAL']))






#r0 = 2.4
#r0 = assumptions['r0']

#INTERVENTIONS = [
#    None, 
#    {
#        datetime.date(2020, 3, 23): 1.3,
#        datetime.date(2020, 4, 20): 1.1,
#        datetime.date(2020, 5, 22): 0.8,
#        datetime.date(2020, 6, 23): r0
#    },
#    {
#        datetime.date(2020, 3, 23): 1.7,
#        datetime.date(2020, 6, 23): r0
#    },
#    {
#        datetime.date(2020, 3, 23): 1.3,
#        datetime.date(2020, 3, 31): 0.3,
#        datetime.date(2020, 4, 28): 0.2,
#        datetime.date(2020, 5,  6): 0.1,
#        datetime.date(2020, 5, 10): 0.35,
#        datetime.date(2020, 5, 18): r0
#    }
#]

#model_state('USA', 'CA', INTERVENTIONS[0])